In [1]:
import pandas as pd
import httpx
import sys
import os

from dotenv import load_dotenv
load_dotenv()

sys.path.append(os.path.abspath('..'))
import config

In [2]:
calls = {}
for year in range(2018,2025):
    calls[year] = pd.read_csv(config.DATA_DIR_RAW / f"pd_calls_for_service_{year}_datasd.csv",
                        dtype=config.COLUMN_DTYPES)

calls[2018].head()

,incident_num,date_time,day_of_week,address_number_primary,address_dir_primary,address_road_primary,address_sfx_primary,address_dir_intersecting,address_road_intersecting,address_sfx_intersecting,call_type,disposition,beat,priority
0,E18010000001,2018-01-01 00:00:09,0,0,NaN,MONROE,AVE,NaN,PARK,NaN,FD,K,624,2
1,E18010000002,2018-01-01 00:00:41,0,0,NaN,MAIN,ST,NaN,THOR,NaN,AU1,W,511,1
2,E18010000003,2018-01-01 00:00:52,0,4300,NaN,OCEAN,BLV,NaN,NaN,NaN,1016,A,122,2
3,E18010000004,2018-01-01 00:00:55,0,4500,NaN,BOYLSTON,ST,NaN,NaN,NaN,AU1,W,445,1
4,E18010000005,2018-01-01 00:00:56,0,0,NaN,04TH,AVE,NaN,ISLAND,NaN,1151,O,523,3


In [3]:
calls_11_7 = pd.DataFrame(columns=calls[2018].columns)
calls_SPT = pd.DataFrame(columns=calls[2018].columns)
for year in calls:
    cur_year_11_7 = calls[year][calls[year]["call_type"] == "11-7"]
    cur_year_SPT = calls[year][calls[year]["call_type"] == "11-7SPT"]

    calls_11_7 = pd.concat([calls_11_7, cur_year_11_7])
    calls_SPT = pd.concat([calls_11_7, cur_year_11_7])

calls_11_7.head()

,incident_num,date_time,day_of_week,address_number_primary,address_dir_primary,address_road_primary,address_sfx_primary,address_dir_intersecting,address_road_intersecting,address_sfx_intersecting,call_type,disposition,beat,priority
56,E18010000057,2018-01-01 00:14:09,0,4800,NaN,GUYMON,ST,NaN,NaN,NaN,11-7,K,445,1
465,E18010000496,2018-01-01 04:51:56,0,4900,NaN,NARRAGANSETT,AVE,NaN,NaN,NaN,11-7,K,614,1
492,E18010000527,2018-01-01 05:48:03,0,18000,NaN,CABELA,DR,NaN,NaN,NaN,11-7,O,234,0
1177,E18010001268,2018-01-01 18:08:17,0,0,NaN,SAN MARCOS,AVE,NaN,LAUREL,NaN,11-7,K,812,1
1214,E18010001311,2018-01-01 18:45:58,0,3700,NaN,HERMAN,AVE,NaN,NaN,NaN,11-7,K,813,1


In [4]:
def get_search_fuzzy(query):
    """
    Perform a fuzzy search for an address or point of interest based on a query string.
    :param query: Query string for the search.
    :return: JSON response from Azure Maps.
    """
    url = "https://atlas.microsoft.com/search/fuzzy/json"
    params = {
        'subscription-key': os.getenv("API_KEY"),
        'api-version': '1.0',
        'query': query
    }

    response = httpx.get(url, params=params)
    return response.json()

In [5]:
query = 'Golden Gate Bridge, San Francisco, CA'
response_json = get_search_fuzzy(query)

for item in response_json['results']:
    print("Result Type:", item["type"])
    if "poi" in item:
        print("Name:", item["poi"]["name"])
    print("Address:", item["address"]["freeformAddress"])
    print("Lat Long:", item["position"], "\n")

Result Type: POI
Name: Golden Gate Bridge Parking
Address: 983 Marine Drive, San Francisco, CA 94129
Lat Long: {'lat': 37.8077, 'lon': -122.4703} 

Result Type: POI
Name: Golden Gate Bridge Views
Address: Golden Gate Bridge South, San Francisco, CA 94129 (Merchant Road)
Lat Long: {'lat': 37.8058, 'lon': -122.4769} 

Result Type: Street
Address: Golden Gate Bridge, San Francisco, CA
Lat Long: {'lat': 37.818325, 'lon': -122.478496} 

Result Type: Street
Address: Golden Gate Bridge South, San Francisco, CA 94129 (Golden Gate Bridge, US Highway 101 South, Pacific Coast Highway)
Lat Long: {'lat': 37.808803, 'lon': -122.477174} 

Result Type: Street
Address: Golden Gate Bridge North, San Francisco, CA 94129
Lat Long: {'lat': 37.80897, 'lon': -122.477137} 

Result Type: POI
Name: Golden Gate Bridge
Address: Merchant Road, San Francisco, CA 94129
Lat Long: {'lat': 37.8063, 'lon': -122.475988} 

Result Type: POI
Name: Golden Gate Bridge Viewpoints
Address: Battery East Road, San Francisco, CA 9